# Chatbot
In this tutorial, we'll be designing a chatbot with the capability to retain information from previous prompts and responses, enabling it to maintain context throughout the conversation. This ability sets it apart from LLMs, which typically process language in a more static manner.

---
## 1.&nbsp; Installations and Settings 🛠️

We additionally install the main langchain package here as we require the memory function from it.

In [ ]:
!pip install -qqq -U langchain-groq
!pip install -qqq -U langchain-huggingface
!pip install -qqq -U langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 3.4 MB/s eta 0:00:00


Again, import our HF Access Token.

In [ ]:
import os
from google.colab import userdata # we stored our access token as a colab secret

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('Hugging')

---
## 2.&nbsp; Setting up your LLM 🧠

[Groq's website](https://groq.com/)

[Langchain's ChatGroq docs](https://python.langchain.com/v0.2/api_reference/groq/chat_models/langchain_groq.chat_models.ChatGroq.html)


In [ ]:
from langchain_groq import ChatGroq
from google.colab import userdata

llm = ChatGroq(temperature=0,
               groq_api_key=userdata.get('groq'),
               model_name="mixtral-8x7b-32768")

### 2.1.&nbsp; Test your LLM

In [ ]:
answer = llm.invoke("Write a poem about Data Science.")
print(answer)

content="In a world where information flows like a river,\nData Science is the skill to uncover its giver.\nWith numbers and figures, patterns emerge,\nA story unfolds, a message to purvey.\n\nThrough the noise and the clutter,\nData Science does not falter.\nWith algorithms and models,\nIt deciphers and makes sense of it all.\n\nFrom the vast and the endless,\nData Science finds the relevant.\nIn the chaos and the confusion,\nData Science brings clarity and truth.\n\nA tool for progress, a beacon of light,\nData Science guides us through the night.\nIn every decision, every action,\nData Science is the foundation.\n\nSo let us celebrate this field,\nWith knowledge and power it's revealed.\nData Science, a marvel to behold,\nA story of data, waiting to be told." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 191, 'prompt_tokens': 15, 'total_tokens': 206, 'completion_time': 0.303867858, 'prompt_time': 0.002049864, 'queue_time': 0.011795354, 'total_time': 0.

In [ ]:
answer_2 = llm.invoke("Tell me a joke.")
print(answer_2)

content="Sure, here's one:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 13, 'total_tokens': 39, 'completion_time': 0.040186114, 'prompt_time': 0.002184194, 'queue_time': 0.012424474, 'total_time': 0.042370308}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None} id='run-b3e746c8-c722-4e47-9b7d-13b335f9dd79-0' usage_metadata={'input_tokens': 13, 'output_tokens': 26, 'total_tokens': 39}


---
## 3.&nbsp; Making a chatbot 💬
To transform a basic LLM into a chatbot, we'll need to infuse it with additional functionalities: prompts, memory, and chains.

**Prompts** are like the instructions you give the chatbot to tell it what to do. Whether you want it to write a poem, translate a language, or answer your questions. They provide the context and purpose for its responses.

**Memory** is like the chatbot's brain. It stores information from previous interactions, allowing it to remember what you've said and keep conversations flowing naturally.

The **chain** is like a road map that guides the conversation along the right path. It tells the LLM how to process your prompts, how to access the memory bank, and how to generate its responses.

In essence, prompts provide the direction, memory retains the context, and chains orchestrate the interactions.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


# Notice that "chat_history" is present in the prompt template
template = """You are a nice chatbot having a conversation with a human. Keep your answers short and succinct.

Previous conversation:
{chat_history}

New human question: {question}
Response:"""

prompt = PromptTemplate.from_template(template)

# Notice that we need to align the `memory_key`
memory = ConversationBufferMemory(memory_key="chat_history")

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory
)

<ipython-input-6-0a4b71591ee4>:18: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")
<ipython-input-6-0a4b71591ee4>:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation = LLMChain(


We can now ask questions of our chatbot.

In [ ]:
conversation({"question": "tell me a joke"})

<ipython-input-7-31d5c5b50afb>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  conversation({"question": "tell me a joke"})


{'question': 'tell me a joke',
 'chat_history': '',
 'text': "Sure, here's a light-hearted joke for you: Why don't scientists trust atoms? Because they make up everything! I hope it brings a smile to your face. Do you have any other questions or topics you'd like to discuss?"}

And we can ask about themes from previous messages.

In [ ]:
conversation({"question": "Explain why that joke was funny."})

{'question': 'Explain why that joke was funny.',
 'chat_history': "Human: tell me a joke\nAI: Sure, here's a light-hearted joke for you: Why don't scientists trust atoms? Because they make up everything! I hope it brings a smile to your face. Do you have any other questions or topics you'd like to discuss?",
 'text': 'The joke is funny because it plays with the concept of atoms being the basic units of matter, and thus "making up" everything in the physical world. The unexpected twist is that atoms can\'t be trusted, which adds a layer of humor to the scientific fact.'}

We can also use our python skills to create a better chatbot experience.

In [ ]:
conversation_2 = LLMChain(
    llm = llm,
    prompt = prompt,
    verbose = False,
    memory = memory
)

# Start the conversation loop
while True:
  user_input = input("You: ")

  # Check for exit condition -> typing 'end' will exit the loop
  if user_input.lower() == 'end':
      print("Ending the conversation. Goodbye!")
      break

  # Get the response from the conversation chain
  response = conversation_2({"question": user_input})

  # Print the chatbot's response
  print("Chatbot:", response["text"])

KeyboardInterrupt: Interrupted by user